In [40]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [41]:
class AITextDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

In [53]:
data = pd.read_pickle("feature_engineered_embeddings")
shuffled = data.sample(frac = 1)
splits = np.array_split(shuffled, 2)

/Users/devammondal/PycharmProjects/lstm/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [67]:
train = splits[0]
test = splits[1]

In [68]:
print(train)

                                                    text  generated  \
26673  Everyday, millions of people around the world ...          1   
23557  There is no definitive answer to this question...          1   
22025  There is no denying that medical care and educ...          1   
26917  We all want to make a positive impact on the w...          1   
16527  DISTANCE LEARNING\n\nI think students can be b...          0   
...                                                  ...        ...   
18705  \nFamously competitive rivals, Cristiano Ronal...          1   
23126  There is no denying that facts are important. ...          1   
19088  \nCaring for someone with mental health issues...          1   
20046  \nHaving a positive attitude in a difficult si...          1   
8272   How to know how a student feels in your class?...          0   

                                              embeddings  
26673  [[tensor(0.1263), tensor(-0.1514), tensor(-0.3...  
23557  [[tensor(0.1713), tens

In [69]:
train_dataset = AITextDataset(data=list(train.embeddings), labels=list(train.generated))
test_dataset = AITextDataset(data=list(test.embeddings), labels=list(train.generated))

In [118]:
class LSTM(torch.nn.Module):
    def __init__(self, input_length, hidden_length, layers_number, output_size):
        super(LSTM, self).__init__()
        self.hidden_length = hidden_length
        self.layers_number = layers_number
        self.lstm = torch.nn.LSTM(input_length, hidden_length, layers_number, batch_first = True)
        self.connected = torch.nn.Linear(hidden_length, output_size)
        
    def forward(self, x):
        hidden_state = torch.zeros(self.layers_number, x.size(0), self.hidden_length)
        memory = torch.zeros(self.layers_number, x.size(0), self.hidden_length)
        out, _  = self.lstm(x, (hidden_state, memory))
        out = self.connected(out[:, -1, :])
        return out

In [119]:
model = LSTM(768, 64, 2, 2)

In [98]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters())

In [124]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=50, shuffle = True)

In [125]:
EPOCHS = 1
train_loss = []
train_acc = []
test_acc = []

for epoch in range(EPOCHS):
    for i, data in enumerate(train_loader):
        tensor, label = data
        prediction = model(tensor)
        print(label)
        print(prediction)
        count = 0 
        for j in range(len(prediction)):
            classification = 0
            if prediction[j][1] > prediction[j][0]:
                classification = 1
            if label[j] == classification:
                count += 1
        accuracy = count / 50
        loss = loss_function(prediction, label)
        train_loss.append(loss)
        train_acc.append(accuracy)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        print(f"epoch {epoch} step {i} loss {loss} accuracy {accuracy}") 
    break

tensor([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        1, 1])
tensor([[-0.0668,  0.1146],
        [-0.0276,  0.1014],
        [-0.0519,  0.0832],
        [-0.0335,  0.1057],
        [-0.0661,  0.0974],
        [ 0.0414,  0.1124],
        [-0.0213,  0.1118],
        [-0.0706,  0.1017],
        [ 0.0438,  0.1020],
        [-0.0068,  0.1327],
        [-0.0673,  0.0797],
        [ 0.0787,  0.1168],
        [-0.0151,  0.1075],
        [-0.0426,  0.1114],
        [-0.0503,  0.1005],
        [ 0.0952,  0.1411],
        [ 0.0740,  0.1202],
        [-0.0578,  0.1047],
        [-0.0351,  0.1131],
        [-0.0414,  0.1103],
        [ 0.0643,  0.1165],
        [-0.0311,  0.1031],
        [-0.0663,  0.1288],
        [-0.0487,  0.0888],
        [-0.0736,  0.1313],
        [ 0.0232,  0.1055],
        [-0.0443,  0.1196],
        [-0.0341,  0.0788],
        [ 0.0183,  0.1263],
        [-0.0